# ChatGPT Coding Example with Python

## Welcome
Welcome to the ChatGPT Coding Example with Python notebook. This file is used to showcase some methods used to interact with Large Language Models (LLMs) and get productive results. Specifically, we will be using ChatGPT to interact with social media to gather information. In part two, we will look at prompt engineering techniques to elicit specific responses from the model. 

_Note: all code follows PEP 8 standards_



In [1]:
# global imports
import json
import os
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import openai
from pandas import json_normalize

# Constants
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Configure OpenAI API key
openai.api_key = OPENAI_API_KEY


### Social Media Listening 

In the following scenario, a client (Sam's Club) has asked us to build an app that allows us to keep track of what people say about their brand on social media. A core component of this application is based upon AI capabilities to extract information from the interactions. For this example, we will create three components to process X posts (aka Tweets), and extract information about them in three stages:

1. Sentiment and Emotion Analysis: Analyze the predominant sentiment and emotion of each tweet.
2. Topic modelling: Find the common topics discussed.
3. Categorization: Classify the posts into predefined categories (Content Quality, Customer Support, Spam, Membership Issues, Marketing, or Other)

After processing the posts, our solution will create a table with all the information needed to create a report.

### Feature Selection

We have extracted the posts into a JSON file (posts.json). To simplify processing we will eliminate some features to more closely align with our goals. 

In [2]:
# Read JSON data from "posts.json" and convert it to a DataFrame
try:
    with open("posts.json") as file:
        data = json.load(file)
except FileNotFoundError:
    print("File not found. Please check the file name and path.")
    # Handle the exception as needed
except json.JSONDecodeError:
    print("Error decoding JSON. Please check the file content.")
    # Handle the exception as needed
else:
    # Flatten the JSON data into a DataFrame
    df = json_normalize(data)

    # Display the first 5 rows of the DataFrame
    print(df.head())




                       created_at                   id               id_str  \
0  Mon May 15 18:21:47 +0000 2023  1658175862045302797  1658175862045302797   
1  Mon May 15 17:29:18 +0000 2023  1658162654479888397  1658162654479888397   
2  Mon May 15 17:04:13 +0000 2023  1658156342199332864  1658156342199332864   
3  Mon May 15 16:54:52 +0000 2023  1658153988703809536  1658153988703809536   
4  Mon May 15 16:54:24 +0000 2023  1658153870575521793  1658153870575521793   

                                           full_text  truncated  \
0  KitchenAid Mixer Sale on @SamsClub | $90 Off P...      False   
1           @KellieShai_ All we need is some milk. 🍪      False   
2  Happy National Chocolate Chip Day! No denying ...      False   
3  This is heartbreaking that less than a year im...      False   
4  @VIZIOsupport I bought a vizio tv from @SamsCl...      False   

  display_text_range                                             source  \
0           [0, 128]  <a href="https://mobile.t

In [3]:
try:
    # Get a list of all column names in the DataFrame
    all_columns = df.columns.tolist()

    # Define the base column names to retain
    base_columns_to_keep = ['created_at', 'id', 'full_text', 'in_reply_to_screen_name']

    # Add columns that start with 'metadata.' or 'user.' to the list of columns to keep
    additional_columns_to_keep = [col for col in all_columns if col.startswith('metadata.') or col.startswith('user.')]
    columns_to_keep = base_columns_to_keep + additional_columns_to_keep

    # Update the DataFrame to include only the specified columns
    df = df[columns_to_keep]

    # Display the first 5 rows of the DataFrame
    print(df.head())


except NameError:
    print("DataFrame 'df' is not defined. Please ensure it is created and loaded correctly.")
except KeyError as e:
    print(f"Key error: {e}. Please ensure the specified columns exist in the DataFrame.")



                       created_at                   id  \
0  Mon May 15 18:21:47 +0000 2023  1658175862045302797   
1  Mon May 15 17:29:18 +0000 2023  1658162654479888397   
2  Mon May 15 17:04:13 +0000 2023  1658156342199332864   
3  Mon May 15 16:54:52 +0000 2023  1658153988703809536   
4  Mon May 15 16:54:24 +0000 2023  1658153870575521793   

                                           full_text in_reply_to_screen_name  \
0  KitchenAid Mixer Sale on @SamsClub | $90 Off P...                    None   
1           @KellieShai_ All we need is some milk. 🍪             KellieShai_   
2  Happy National Chocolate Chip Day! No denying ...                    None   
3  This is heartbreaking that less than a year im...                Roja1670   
4  @VIZIOsupport I bought a vizio tv from @SamsCl...            VIZIOsupport   

  metadata.iso_language_code metadata.result_type              user.id  \
0                         en               recent             20445389   
1                     

In [4]:
try:
    # Drop columns where all elements are missing and the 'user.withheld_in_countries' column
    df = df.dropna(axis=1, how='all').drop(columns=['user.withheld_in_countries'])

    # Display the first 5 rows of the DataFrame
    print(df.head())


except NameError:
    print("DataFrame 'df' is not defined. Please ensure it is created and loaded correctly.")
except KeyError as e:
    print(f"Key error: {e}. Please ensure the specified columns exist in the DataFrame.")



                       created_at                   id  \
0  Mon May 15 18:21:47 +0000 2023  1658175862045302797   
1  Mon May 15 17:29:18 +0000 2023  1658162654479888397   
2  Mon May 15 17:04:13 +0000 2023  1658156342199332864   
3  Mon May 15 16:54:52 +0000 2023  1658153988703809536   
4  Mon May 15 16:54:24 +0000 2023  1658153870575521793   

                                           full_text in_reply_to_screen_name  \
0  KitchenAid Mixer Sale on @SamsClub | $90 Off P...                    None   
1           @KellieShai_ All we need is some milk. 🍪             KellieShai_   
2  Happy National Chocolate Chip Day! No denying ...                    None   
3  This is heartbreaking that less than a year im...                Roja1670   
4  @VIZIOsupport I bought a vizio tv from @SamsCl...            VIZIOsupport   

  metadata.iso_language_code metadata.result_type              user.id  \
0                         en               recent             20445389   
1                     

### Sentiment and Emotion Analysis

Define our function for sentiment and emotion analysis. 

For each post, we want to find out the predominant sentiment: Positive(1), Neutral(0), Negative(-1).

Also, we will provide a more nuanced value to represent the emotion: Joy(1), Surprise(0.80), Neutral(0.60), Sadness(0.40), Mistrust(0.20), and Disgust(0).

_Note: Keep in mind that, prompts can handle a fixed amount of tokens so we should check that we're not exceeding the amount of tokens when creating our function._




In [5]:

def sentiment_emotion_analysis(df):
    """
    Perform sentiment and emotion analysis on each post in a DataFrame.

    This function iterates through each post contained in the 'full_text' column of the input DataFrame.
    For each post, it constructs a prompt and sends it to the OpenAI API to perform sentiment analysis
    and emotion detection. The sentiment is categorized as Positive (1), Neutral (0), or Negative (-1).
    Additionally, the emotion is represented by a nuanced value: Joy (1), Surprise (0.80), Neutral (0.60),
    Sadness (0.40), Mistrust (0.20), and Disgust (0). The function handles JSON decoding errors and
    general exceptions during the API response parsing and logs these errors.

    The results of the sentiment and emotion analysis are appended to the DataFrame as new columns,
    namely 'sentiment', 'sentiment_value', 'emotion', and 'emotion_value', where each entry corresponds
    to the analysis results of the respective post.

    Parameters:
    df (pandas.DataFrame): A pandas DataFrame containing a column 'full_text' with posts to analyze.

    Returns:
    pandas.DataFrame: The modified DataFrame with added columns for sentiment ('sentiment', 'sentiment_value')
    and emotion ('emotion', 'emotion_value') analysis results.

    Exceptions:
    - JSONDecodeError: Handles exceptions related to JSON decoding errors in the API response.
    - Exception: Catches and logs any other general exceptions that may occur during the API call or response parsing.

    Note:
    The function requires an active OpenAI API key set up in the environment to perform the analysis.
    """
    sentiments = []
    emotions = []

    for post in df['full_text']:

        try:
            response = openai.chat.completions.create(
                model="gpt-4-turbo-preview",
                response_format={ "type": "json_object" },
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that looks at posts (aka tweets), then determines sentiment and emotion. Analyze the sentiment of the following posts and categorize them as Positive (1), Neutral (0), or Negative (-1). Also, assign an emotion to the post, with a corresponding value: Joy (1), Surprise (0.80), Neutral (0.60),Sadness (0.40), Mistrust (0.20), and Disgust (0). Return the analysis in a JSON format, consisting solely of the 'sentiment' and 'emotion' keys with their respective values('sentiment_value' and 'emotion_value'). Exclude all other information or text that is not part of the JSON format."},
                    {"role": "user", "content": f" Post: {post}"},
                ],
                temperature=0,
                max_tokens=100,
                top_p=0.50,
                frequency_penalty=0.0,
                presence_penalty=0.0
            )
            
            response_text = response.choices[0].message.content.strip()
        

            # Check if response text is in JSON format
            if response_text.startswith('{') and response_text.endswith('}'):
                response_json = json.loads(response_text)
                sentiments.append(response_json["sentiment"])
                emotions.append(response_json["emotion"])
            else:
                logging.error(f"Invalid JSON format for post: {post}")
                sentiments.append(("Invalid Format", 0))
                emotions.append(("Invalid Format", 0))

        except json.JSONDecodeError as json_error:
            logging.error(f"JSON decoding error for post: {post}, Error: {json_error}")
            sentiments.append(("Error", 0))
            emotions.append(("Error", 0))
        

    df['sentiment'] = sentiments
    df['emotion'] = emotions

    return df


### Topic Modeling

Define our function for topic extraction. For each post find out the broader topic discussed. 

In [6]:
# Define the get_topic function to determine the topic of a post
def get_topic(post):
    """
    Determines the topic of a post (tweet) using the OpenAI API.
    The topic is derived in one or two words and returned in sentence case.
    """
    response = openai.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that looks at posts (aka tweets), then determines the topic or subject in one or two words. The topic should be broad enough to cover a large number of posts, but specific enough to be useful. Put all topics in sentence case and don't include any punctuation at the end of the topic. Respond with: The topic of this post is:<topic>"},
            {"role": "user", "content": f"Post: {post}"}
        ],
        temperature=0.3,
        max_tokens=60,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    
    response_text = response.choices[0].message.content.strip().replace("The topic of this post is: ", "")
    return response_text



### Generate Results

Generate our sentiment, emotion, and topic and merge it into our dataframe

In [7]:
# Analyze sentiment and emotion
df = sentiment_emotion_analysis(df)

# Determine topic for each row in 'full_text'
df['topic'] = df['full_text'].apply(get_topic)

# Save to CSV without index
df.to_csv('sentiment_emotion_topic.csv', index=False)

# Display first 5 rows
print(df.head())



                       created_at                   id  \
0  Mon May 15 18:21:47 +0000 2023  1658175862045302797   
1  Mon May 15 17:29:18 +0000 2023  1658162654479888397   
2  Mon May 15 17:04:13 +0000 2023  1658156342199332864   
3  Mon May 15 16:54:52 +0000 2023  1658153988703809536   
4  Mon May 15 16:54:24 +0000 2023  1658153870575521793   

                                           full_text in_reply_to_screen_name  \
0  KitchenAid Mixer Sale on @SamsClub | $90 Off P...                    None   
1           @KellieShai_ All we need is some milk. 🍪             KellieShai_   
2  Happy National Chocolate Chip Day! No denying ...                    None   
3  This is heartbreaking that less than a year im...                Roja1670   
4  @VIZIOsupport I bought a vizio tv from @SamsCl...            VIZIOsupport   

  metadata.iso_language_code metadata.result_type              user.id  \
0                         en               recent             20445389   
1                     

### Categorization

Next we will categorize each post based on it's content into one of the following categories: 
Content Quality, Customer Support, Spam, Membership issues, Marketing and Other. 

For each post, we will calculate the category and a score to show the relevancy of the post to the category using the function below.

In [8]:

def categorize_post(text):
    """
    Categorizes a post into predefined categories using an AI model.

    Args:
        text (str): The post text to categorize.

    Returns:
        tuple: A tuple containing the category (str) and score (float).
    """
    prompt = f"""
    Given the following post, please categorize it into one of the following categories with a score from 0 (not relevant) to 1 (highly relevant) and all values in between based on how relevant the post is in relation to the category. 

    Examples:
    
    Tweet: Finna go to sams club and get a box of nature valley bars and open em in this bed
    Answer: Category: Other, Score: 0.5

    Tweet: 🚨 BRAND NEW JUNE @SamsClub 2023 VIDEO 👉🏼
    Answer: Category: Marketing, Score: 1

    Please provide the category and score in the following JSON format: {{"category": <category>, "score": <score>}}
    
    The categories are: 

    - Content Quality
    - Customer Support
    - Spam
    - Membership Issues
    - Marketing
    - Other

    """

    try:
        response = openai.chat.completions.create(
            model="gpt-4-turbo-preview",  
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"Post:{text}"}
            ],
            response_format={ "type": "json_object" },
            temperature=0.3, 
            max_tokens=60,  
            top_p=1.0,  
            frequency_penalty=0.0, 
            presence_penalty=0.0  
        )
    
        result_dict = json.loads(response.choices[0].message.content.strip())
        category = result_dict["category"]
        score = float(result_dict["score"])
        return category, score


    except json.JSONDecodeError:
        raise ValueError(f"Failed to decode the response into JSON format: {response}")


In [9]:
# Apply the categorize_post function to each item in the 'full_text' column.
# Save the results to new columns 'category' and 'score'.
categories_and_scores = df['full_text'].map(categorize_post)
df['category'], df['score'] = zip(*categories_and_scores)

# Attempt to save the processed data to a CSV file and handle potential errors.
try:
    df.to_csv('category.csv', index=False)
except IOError as e:
    print(f"Error saving to CSV: {e}")


### Final Cleanup
Build the final data that will be used to analyze and create visualizations from the Social Media interactions.

In [10]:
# Show the current column list
print(df.columns)

# Refactor column name transformations
df.columns = df.columns.str.replace('.', '_', regex=False).str.lower()
df = df.rename(columns={'id': 'post_id'})

# Save to CSV
df.to_csv('finaloutput.csv', index=False)

# Print the updated columns to verify changes
print(df.columns)


Index(['created_at', 'id', 'full_text', 'in_reply_to_screen_name',
       'metadata.iso_language_code', 'metadata.result_type', 'user.id',
       'user.id_str', 'user.name', 'user.screen_name', 'user.location',
       'user.description', 'user.url', 'user.entities.url.urls',
       'user.entities.description.urls', 'user.protected',
       'user.followers_count', 'user.friends_count', 'user.listed_count',
       'user.created_at', 'user.favourites_count', 'user.geo_enabled',
       'user.verified', 'user.statuses_count', 'user.contributors_enabled',
       'user.is_translator', 'user.is_translation_enabled',
       'user.profile_background_color', 'user.profile_background_image_url',
       'user.profile_background_image_url_https',
       'user.profile_background_tile', 'user.profile_image_url',
       'user.profile_image_url_https', 'user.profile_banner_url',
       'user.profile_link_color', 'user.profile_sidebar_border_color',
       'user.profile_sidebar_fill_color', 'user.profile

## Prompt Design / Engineering

### Story Generation

Our first task is to write a prompt to ask the LLM to create a kids short story. We will have the AI output an HTML table with the names of the characters from the story and their role in the story as well. 

First, we create a function to run our prompt and get our story.

In [11]:
def generate_story(prompt):
    """
    Generates a story based on the given prompt using an AI model.

    Args:
        prompt (str): The prompt to generate the story from.

    Returns:
        str: The generated story content.
    """
    return openai.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[{"role": "user", "content": prompt}],
        temperature=1,
        max_tokens=1000,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    ).choices[0].message.content



Now it's just a simple matter of calling the function with our prompt to get our results

In [12]:

prompt = """
# CONTEXT #
Craft a children's story in the style of Hans Christian Andersen. 

# OBJECTIVE #
1. Word Count: Ensure the story is no more than 500 words.
2. Opening: Start with an imaginative scene to captivate the reader.
3. Setting: Include time of day, weather, and immersive details.
4. Minor Characters: Introduce them with memorable details.
5. Main Character: Describe their appearance, personality, and motivations.
6. Plot and Resolution: Create a clear, satisfying story with child-appropriate themes.
7. Language: Use vivid imagery and language to engage the reader.

# STYLE #
Hans Christian Andersen's storytelling style.

# TONE #
Enchanting and appropriate for young readers.

# AUDIENCE #
Children and those who enjoy fairy tales.

# RESPONSE: Story and HTML Table  #
Story: 
<Insert crafted story here>

Characters and Roles: 
<Insert HTML table here listing characters and their roles in the story>

"""

# Call our function to generate the story
response_text = generate_story(prompt)
print(response_text)

# Writing to a file
with open("StoryGeneration.txt", "w") as file:
    file.write(response_text)



Story:

Once upon a time, in a land where the sun dipped low, painting the sky in shades of orange and pink, there stood a little village surrounded by vast, whispering forests. The time was twilight; that magical hour when everything seems possible and fairy tales come to life. In this village, nestled at the foot of a slumbering mountain, lived a boy named Wilhelm.

Wilhelm was no ordinary boy; his eyes sparkled with a curiosity that matched the stars, and his laughter was as infectious as the melody of spring. His hair was a mess of tangles, resembling the wild brambles of the woods, and his heart, pure and kind, yearned for adventure beyond the mountains that cradled his home.

Among the villagers, two individuals stood out for their wisdom and kindness: Old Martha, a baker whose hands crafted breads that could soothe even the most troubled soul, and Master Henrik, a clockmaker whose timepieces were said to tick in harmony with the heartbeat of the earth. They adored Wilhelm and of

### Story Summarization

Based on the story created by the LLM in the previous task, we will make a summaries. This will consist of two versions: formal and an informal. We will make sure to return and store both versions.

In [13]:

# Split the story text to extract the relevant part
story = response_text.split('Characters and Roles:')[0].strip()

# Create formal summary
prompt_formal = f"""
# CONTEXT #
You are an expert in story summary writing. Your task is to provide a formal summary of a given story.

# OBJECTIVE #
Produce a concise, professional summary suitable for publication. Focus on summarizing the story's content and extracting key information and arguments. The summary should be comprehensive, detailed, yet concise, limited to no more than 100 words.

# STYLE #
Professional, formal

# TONE #
Objective, Clear

# AUDIENCE #
Readers seeking a succinct overview of the story for publication purposes.

# RESPONSE FORMAT #
Provide the summary in the following structure:

Formal Summary: [Insert summary here]

# STORY TEXT #
{story}

"""

response_formal = openai.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[{"role": "user", "content": prompt_formal}],
        temperature=0.10,
        max_tokens=200,
        top_p=0.50,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )



formal_summary = response_formal.choices[0].message.content

print(formal_summary + "\n\n")


# Create informal summary
prompt_informal = f"""
# CONTEXT #
You're taking on the role of a casual, yet expert, story summarizer.

# OBJECTIVE #
Craft a fun, informal summary of the provided story, as if you're chatting with a good friend. Your goal is to cover the story's main points and key details in an engaging way. Keep it light, lively, and no more than 100 words.

# STYLE #
Conversational, like you're explaining to a friend

# TONE #
Lighthearted, Engaging

# AUDIENCE #
A good friend interested in a quick, entertaining rundown of the story.

# RESPONSE FORMAT #
Deliver the summary as follows:

Informal Summary: [Insert summary here]

# STORY TEXT #
{story}

"""

response_informal = openai.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[{"role": "user", "content": prompt_informal}],
        temperature=0.80,
        max_tokens=200,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )

informal_summary = response_informal.choices[0].message.content

print(informal_summary +"\n")

# Save the responses to a text file
with open("Summarization.txt", "w") as file:
    file.write("Formal Summary:\n" + formal_summary + "\n\nInformal Summary:\n" + informal_summary)


Formal Summary: In a magical twilight village, young Wilhelm, known for his curiosity and kind heart, embarks on a quest to find the legendary Glowing Mushroom in the surrounding forest, inspired by tales from beloved villagers Old Martha and Master Henrik. Guided by a wise fox, Wilhelm discovers the mushroom, only to learn that its true value lies not in possession but in the journey and connections made. Returning home enlightened, Wilhelm's adventure becomes a village legend, celebrating the courage and magic inherent in the pursuit of dreams.


Informal Summary: So, this kid Wilhelm from a postcard-perfect village decides to go full-on adventurer mode to find this mythical Glowing Mushroom, right? Think of him as your classic, wide-eyed protagonist with a dash of Indiana Jones. He's got this squad of village VIPs, like a magical baker and a clockmaker, hyping him up with old tales. Into the spooky woods he goes, teams up with a sassy red fox, and bam! Finds the mushroom. Plot twist

### Translation

Now we will translate the summary into three languages of your preference. The result will be stored in JSON format where keys are the languages for consumption later on.

In [14]:
languages = ['Spanish', 'French', 'German']
formal_translations = {}
informal_translations = {}

for language in languages:
    # Translate formal summary
    prompt_formal = (
        f"Translate the following English text to {language}:\n\n"
        f"{formal_summary}"
    )
    
    response_formal = openai.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[{"role": "user", "content": prompt_formal}],
        temperature=0,
        max_tokens=200,
        top_p=0.30,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )

    formal_translations[language] = response_formal.choices[0].message.content

    # Translate informal summary
    prompt_informal = (
        f"Translate the following English text to {language}:\n\n"
        f"{informal_summary}"
    )

    response_informal = openai.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[{"role": "user", "content": prompt_informal}],
        temperature=0,
        max_tokens=200,
        top_p=0.30,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    
    informal_translations[language] = response_informal.choices[0].message.content

# Store results in JSON format
translations_json = json.dumps({
    "formal_translations": formal_translations,
    "informal_translations": informal_translations
}, ensure_ascii=False, indent=4)

# Write JSON string to a file
with open('translations.json', 'w', encoding='utf-8') as f:
    f.write(translations_json)


### Customer Service Automation

For our last example, we will examine customer service response automation. Based on a product review, we will ask the LLM to write a customer service email addressing the client's review. We should be careful to take into account the clients sentiment when writing a response. Additionally, we will also create action items as suggestions for the company to take based on the review.

In [15]:

# Product review from the customer
review = """
Product Review: Z-Blend Master Blender System

I recently had the opportunity to try out the Z-Blend Master Blender System and I'd like to share my detailed review of this product. During the month of November, I was thrilled to discover that the 17-piece system was on a seasonal sale, priced at a mere $49, which was nearly half off the original price. However, my excitement was short-lived when, around the second week of December, the prices skyrocketed to around $70-$89 for the same system, leaving me a bit puzzled and skeptical about the sudden price increase.

In terms of design and build quality, the Z-Blend Master Blender System has an overall appealing look. However, upon closer examination, I noticed that the part where the blade locks into place doesn't appear as robust as in previous editions released a few years ago. Despite this observation, I intend to handle the blender with utmost care, especially since I use it to crush hard items like beans, ice, and rice. My blending technique involves initially pulverizing these items in the blender before switching to the whipping blade to achieve a finer flour consistency. Similarly, when preparing smoothies, I utilize the cross-cutting blade initially and then switch to the flat blade if I prefer a smoother and less pulpy texture.

For those who enjoy making smoothies, I have a helpful tip to share. It's best to finely cut and freeze the fruits and vegetables before blending them. This way, you can reduce the amount of ice needed while still achieving a delicious and refreshing smoothie. In the case of making sorbet, I've found that using a small to medium-sized food processor yields excellent results.

After approximately a year of regular use, I did encounter a minor issue with the Z-Blend Master Blender System. The motor started making an unusual noise, prompting me to reach out to customer service. Unfortunately, I discovered that the warranty had already expired, requiring me to purchase a new blender. It's worth noting that the overall quality of similar products in the market seems to have declined over time. Manufacturers seem to rely on brand recognition and consumer loyalty to maintain sales, rather than consistently improving the product's durability.

On a positive note, the delivery service for the Z-Blend Master Blender System was commendable. I received the product within just two days of placing my order, which exceeded my expectations and demonstrated excellent customer service.

In conclusion, the Z-Blend Master Blender System offers satisfactory performance and functionality, despite some concerns about the build quality. The fluctuating prices during the sales period are a noteworthy aspect to consider, as it raises questions about fair pricing practices. Given my experience with the motor issue and the general decline in product quality, I recommend potential buyers thoroughly evaluate their options before making a purchase decision.
"""

# Main prompt to get our response
prompt = f"""
# CONTEXT #
Act as a customer service agent responding to a product review given by a customer.

# OBJECTIVE #
1. Craft a response to the customer's review.
2. Generate a bulleted list of actionable steps for the company based on the review's content.

# STYLE #
Professional and friendly with an empathetic tone.

# TONE #
Understanding and reassuring, aiming to placate the customer.

# AUDIENCE #
The primary audience is the customer who wrote the review. The secondary audience is the company's internal team.

# RESPONSE FORMAT #
1. Customer Response: A composed response in 2-3 paragraphs addressing the customer's concerns expressed in the review.
2. Action Items: A list of 2-5 bullet points outlining steps for internal company action.

Response:
<Compose the response to the customer here, adhering to the specified style and tone>

Action Items:
<Bullet list of 2-5 action items for internal company use based on the review's issues>

# INSTRUCTIONS #
- Analyze the customer's review.
- Draft a response that acknowledges the customer's experience, offers empathy, and suggests a resolution or next step.
- Identify key issues in the review and create actionable steps for the company to address these issues.

{review}


"""

# Obtain completion from the model
response = openai.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.6,
    max_tokens=2000,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0
)

# Extract the response text
response_text = response.choices[0].message.content

# Find the start index of the Action Items section
action_items_start = response_text.find("Action Items:")

# Extract the response to the customer
response_to_customer = response_text[:action_items_start].strip().replace("Response:", "").strip()

# Extract the action items
action_items = response_text[action_items_start:].replace("Action Items:", "").strip()

# Print the extracted sections
print("Response:\n", response_to_customer, "\n")
print("Action Items:\n", action_items, "\n")

# Write the response and action items to a text file
with open("TextAutomation.txt", "w") as file:
    file.write(f"Response:\n{response_to_customer}\n\nAction Items:\n{action_items}\n")



Response:
 Customer 
Dear [Customer's Name],

Thank you very much for taking the time to share your detailed review of the Z-Blend Master Blender System. We truly value your feedback and are committed to ensuring our customers have a positive experience with our products. We understand your concerns regarding the price fluctuation and the issues you encountered with the build quality and durability of the blender. It's disappointing to hear that your excitement was dampened by these experiences, and we sincerely apologize for any inconvenience this may have caused.

We appreciate your tips for making smoothies and your overall positive comments about the design and delivery service. It’s insights like yours that help us improve and strive to deliver the best possible products and services. Regarding the motor noise issue you faced, we regret to hear that it occurred just after the warranty period. We would like to learn more about this and see how we can assist you further. Please cont